<a href="https://colab.research.google.com/github/mohaaliothman/Digit-Clustering/blob/main/Copy_of_NLP_Ass2_AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step 1: Install and Import Libraries**

In [ ]:
!pip install transformers datasets seqeval evaluate pandas openpyxl --quiet
!pip install arabert --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 11.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments
)
import evaluate

# **Step 2: Load the Excel File and Preprocess**

In [ ]:
# Read the Excel File
filename = "/content/All_data.xlsx"
df = pd.read_excel(filename, sheet_name="price")

# Preprocess Data
df["Sentence #"].fillna(method="ffill", inplace=True)

grouped = df.groupby("Sentence #").agg({
    "word": list,
    "aspect": list
}).reset_index()

grouped = grouped.rename(columns={"word": "tokens", "aspect": "ner_tags"})

# Encode labels
label_list = sorted(set(tag for tags in grouped["ner_tags"] for tag in tags))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

# Map labels to IDs
grouped["ner_tags"] = grouped["ner_tags"].apply(lambda tags: [label2id[tag] for tag in tags])


dataset = Dataset.from_pandas(grouped)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
dataset_dict = DatasetDict({
    "train": dataset["train"],
    "test": dataset["test"]
})




/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
<ipython-input-3-483a1a6f71cd>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Sentence #"].fillna(method="ffill", inplace=True)
<ipython-input-3-483a1a6f71cd>:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Sentence #"].fillna(method="ffill", inplace=True)


In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['Sentence #', 'tokens', 'ner_tags'],
        num_rows: 3876
    })
    test: Dataset({
        features: ['Sentence #', 'tokens', 'ner_tags'],
        num_rows: 970
    })
})

In [ ]:
grouped["ner_tags"]

,ner_tags
0,"[2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, ..."
1,"[0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, ..."
2,"[2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 1, 2, 2, ..."
3,"[2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, ..."
4,"[2, 2, 2, 1, 2, 0, 0]"
...,...
4841,"[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, ..."
4842,"[2, 2, 0, 2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 1, ..."
4843,"[0, 1, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 0, 1, ..."
4844,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, ..."


In [ ]:
ner_feature = dataset_dict["train"].features["ner_tags"]
ner_feature

Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)

# **Step 3: Tokenize and Align Labels**

In [ ]:
model_checkpoint = "aubmindlab/bert-base-arabertv2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenize and Align Labels
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            if label % 2 == 1:  # Convert B- to I- for same word pieces
                label += 1
            new_labels.append(label)
    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding='max_length',
        max_length=128,
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(tokenize_and_align_labels, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/720k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3876 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

# **Step 4: Define Data Collator and Metric**

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_preds = [[id2label[p] for (p, l) in zip(pred, label) if l != -100]
                  for pred, label in zip(predictions, labels)]
    results = metric.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

# **Step 5: Load Model**

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **Step 6: Define Trainer**

In [ ]:
training_args = TrainingArguments(
    output_dir="arabic-modernbert-absa",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-10-15da9642b697>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# **Step 7: Train and Evaluate**

In [ ]:
trainer.train()
trainer.evaluate()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.354400,0.349312,0.668603,0.555308,0.606712,0.875459
2,0.288900,0.317071,0.716800,0.597068,0.651478,0.887889
3,0.259100,0.314941,0.696727,0.612287,0.651783,0.887430


{'eval_loss': 0.31494104862213135,
 'eval_precision': 0.6967265047518479,
 'eval_recall': 0.612286562731997,
 'eval_f1': 0.6517830682603971,
 'eval_accuracy': 0.8874295860886603,
 'eval_runtime': 7.1378,
 'eval_samples_per_second': 135.897,
 'eval_steps_per_second': 8.546,
 'epoch': 3.0}

# **Step 8: Test on New Sentences (Optional)**

In [ ]:
def predict_sentence(sentence):
    model.eval()
    words = sentence.split()
    inputs = tokenizer(words, return_tensors="pt", is_split_into_words=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)[0]

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    prediction_labels = [id2label[p.item()] for p in predictions]

    print("\n Prediction Results:")
    for token, label in zip(tokens, prediction_labels):
        if token not in tokenizer.all_special_tokens:
            print(f"{token:15} -> {label}")

# Example usage
predict_sentence("الأكل كان ممتاز لكن خدمة التوصيل سيئة")
predict_sentence("المكان كان جيد لكن الاستقبال سيء")





 Prediction Results:
الأ             -> O
##كل            -> O
كان             -> O
ممتاز           -> I
لكن             -> O
خدم             -> B
##ة             -> B
التو            -> O
##صيل           -> O
سيئ             -> I
##ة             -> O

 Prediction Results:
الم             -> B
##كان           -> O
كان             -> O
جيد             -> I
لكن             -> O
الا             -> I
##ست            -> O
##قبال          -> O
سيء             -> I
